In [2]:
#연습문제 9-7 네이버에서 특정 키워드로 관련 장소나 업체정보 수집하기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt 
import random
import os
import pyautogui
import requests


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.

print("=" *80)
print("연습문제 9-7 네이버에서 특정 키워드로 관련 장소나 업체정보 수집하기")
print("=" *80)

query_txt = input('1.크롤링할 키워드는 무엇입니까?(예:강남역 맛집): ')
cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 10)

f_dir=input('3.파일이 저장될 경로만 쓰세요:(예:c:\\temp\\): ')
print("\n")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)
f_name=s+'-'+query_txt+'.txt'
ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

url = 'https://store.naver.com/restaurants/list?filterId=s13479290&query='+query_txt
driver.get(url)

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택


#Step 8: 사용자 요청 건수가 실제 검색 건수보다 많을 경우
# 실제 검색 건수로 리셋하기


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result= soup.find('div', class_='pagination').find('em',class_='count').get_text( )
result2 = result.replace(",","")
search_cnt = int(result2)

if cnt > search_cnt :
        cnt = search_cnt
print("-" *80)
print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 검색 건수",cnt)

page_cnt = math.ceil(cnt / 20)
print("-" *80)
print("\n")

# 페이지 번호 만들기
page_no=[]

for i in range(5,cnt) :
        if i % 5 == 0 :
                page_no.append(i + 1)

#Step 5. 출력할 업체의 url 을 목록으로 저장하기

now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d:%02d:%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min, now1.tm_sec)

current_page = 1
click_count = 2
count = 0
item=[]
url_cnt = 1

while ( click_count <= page_cnt+1 )  :
        #print("마우스 스크롤 해서 화면을 이동합니다")
        #pyautogui.click(236, 460)
        #pyautogui.scroll(-4000)
        time.sleep(1)
        
        
        html = driver.page_source
        

        soup = BeautifulSoup(html, 'html.parser')
        
        search_result = soup.find('div', class_='list_area').find('ul')
        
        company_list = search_result.select('li a') 
       
        print("전체 %s 페이지 중 현재 %s 번째 페이지의 목록을 추출하는 중입니다" %(page_cnt,current_page))
        for i in company_list :
                
                url = i.get('href')
                time.sleep(0.5)
                item.append(url)
                
                url_cnt += 1

                if url_cnt > cnt*2 :
                    break
        
        i = math.floor(current_page/5)-1
        
        if (cnt > 20 and current_page < page_cnt):
            current_page += 1
            if  current_page == page_no[i] :
                driver.find_element_by_xpath("""//*[@id="container"]/div[2]/div[1]/div/div[2]/div/div[2]/a[7]""").click()
            else :
                driver.find_element_by_link_text("""%s""" %click_count).click()

       
        #current_page += 1
        click_count += 1
        #driver.find_element_by_xpath('''//*[@id="main_pack"]/div[4]/a[10]''').click() #다음페이지클릭
        print("\n")
                                        
     
item2=list(set(item)) # 중복값 제거하기
#print("중복값 제거후 수집된 데이터 건수: ",len(item2))
print("\n")

# 아래 반복문은 수집된 URL 주소를 출력하는 코드인데 잘 동작하지만 시간이 오래 걸려 주석처리함
#cnt2 = 0
#cnt3 = 1
#for i in item2 :
#    print("%s 번째 URL 을 출력합니다========" %cnt3)
#    print(item2[cnt2])
#    time.sleep(1)
#    print("\n")
#    cnt2 += 1
#    cnt3 += 1

# 출력 리스트의 세부 내역을 출력하기

c_name2=[]
intro2=[]
address2=[]
tel2=[]
part2=[]
homepage2=[]
tv2=[]
total_score2=[] 
blog_cnt4=[]

x = 0
url_cnt=1
company_count = 1

while ( url_cnt <= len(item2) ) :

        
            driver.get(item2[x])
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
        
          
            f = open(f_name, 'a',encoding='UTF-8')
            
            try :
              c_name = soup.find('div','biz_name_area').find('strong','name').get_text()
            except AttributeError :
              continue
            else :
               print("%s 번째 업체 정보입니다================ " %company_count)
               print("-" *70)
               print("1.업체명:",c_name)
               company_count += 1
               f.write("-----------------------------------------------------"+"\n")
               f.write('1.업체명:'+ c_name + "\n")

            try :
              intro = soup.find('div', class_='info_inner').find('span','txt').get_text()
            except AttributeError :
              intro = ' '
              print('2.간단소개:',intro)
              f.write('2.간단소개:'+ intro + "\n")
            else :
              print('2.간단소개:',intro)
              f.write('2.간단소개:'+ intro + "\n")
            
            try :
              address = soup.find('span', class_='addr').get_text()
            except AttributeError :
              address = " "
              print("3.도로명주소: "+ address +"\n")
              f.write('3.도로명주소:' + address + "\n")
            else :
              print("3.도로명주소:", address.replace("\n",""))
              f.write('3.도로명주소:' + address.replace("\n","") + "\n")

            #try :
            #  address = soup.find('div','list_item list_item_address').find_all('ul', class_='list_address').get_text()
            #except AttributeError :
            #  continue
            #else :
            #  print("3.도로명주소:", address[0].replace("\n",""))
            #  print("4.지번 주소:", address[1].replace("\n",""))
            #  f.write('3.도로명주소:' + address[0].replace("\n","") + "\n")
              
              
            try :
              tel = soup.find('div', class_='list_item list_item_biztel').find('div','txt').get_text()
            except AttributeError :
              tel = " "
              print(tel)
              f.write('4.연락처:'+ tel+ "\n")
            else :
              print("4.연락처:", tel.strip())
              f.write('4.연락처:'+ tel.strip()+ "\n")

            try :
              part = soup.find('div',class_='biz_name_area').find('span','category').get_text()
            except AttributeError :
              part=" "
              print("5.업종:",part)
              f.write('5.업종:'+ part+"\n")
            else :
              print('5.업종:',part.replace("\n",""))
              f.write('5.업종:'+ part.replace("\n","")+"\n")

            #open_time = soup.find_all('div',class_='biztime_area list_more_view').find('div','biztime ').get_text()
            #p_time = open_time[0].get_text
            #w_time = open_time[1].get_text
            #print('6.평일:',p_time,'주말:',w_time)
            #f.write('6.평일:'+ p_time.replace("\n","")+'주말:'+w_time.replace("\n","")+"\n")

            try :
              homepage=soup.find('div',class_='list_item list_item_homepage').find('a','biz_url').get('href')
            except AttributeError :
              homepage=' '
              print('6.홈페이지:',homepage)
              f.write('6.홈페이지:'+ homepage +"\n")
            else :
              print('6.홈페이지:',homepage)
              f.write('6.홈페이지:'+ homepage +"\n")
            
            try :
              tv = soup.find('div',class_='tv').get_text()
            except AttributeError :
              tv = ' '
              print("7.언론소개:",tv)
              f.write('7.언론소개:'+ tv+"\n")
            else :
              print('7.언론소개:',tv)
              f.write('7.언론소개:'+ tv+"\n")

            try:
              score = soup.find('div',class_='star_area').find('em').get_text()
            except AttributeError :
              total_score = ' '
              print('8.평점:',total_score)
              f.write('8.평점:'+ total_score +"\n")
            else :
              total_score = ('%s / 5' %score)
              print('8.평점',total_score)
              f.write('8.평점:'+ total_score +"\n")

            try :
              blog_cnt = soup.find('div','info_inner').find('a','link').get_text()
            except AttributeError :
              blog_cnt3 = '0'
              print('9.블로그 리뷰 수:', blog_cnt3)
              f.write('9.블로그 리뷰 수:'+ blog_cnt3 +"\n")
            else :
              blog_cnt2 = re.search("\d+",blog_cnt)
              blog_cnt3 = int(blog_cnt2.group())
              print('9.블로그 리뷰 수:', blog_cnt3)
              f.write('9.블로그 리뷰 수:'+ str(blog_cnt3) +"\n")
              
            f.write("-----------------------------------------------------"+"\n")
            f.close()

            url_cnt += 1
            x += 1
              
         
            print("-" *70)
            print("\n")
            
            time.sleep(0.1)

            c_name2.append(c_name)
            intro2.append(intro)
            address2.append(address)
            tel2.append(tel)
            part2.append(part)
            homepage2.append(homepage)
            tv2.append(tv)
            total_score2.append(total_score)
            blog_cnt4.append(str(blog_cnt3))

now2 = time.localtime()
s2 = '%04d-%02d-%02d-%02d-%02d-%02d' % (now2.tm_year, now2.tm_mon, now2.tm_mday, now2.tm_hour, now2.tm_min, now2.tm_sec)

url_cnt -= 1

e_time = time.time( )
t_time = e_time - s_time
t_time2 = round(t_time,1)

print("총 소요시간은 %s 초 입니다 " %t_time2)
print("총 저장 건수는 %s 건 입니다 " %url_cnt)
print("txt파일 저장 완료: 파일명 : %s " %ff_name)



orig_stdout = sys.stdout
f = open(f_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("크롤링 시작 시간은 %s 이고," %s1)
print("크롤링 종료 시간은 %s 이며," %s2)
print("총 소요시간은 %s 초 이며," %t_time2)
print("총 저장 건수는 %s 건 입니다 " %url_cnt)
print("=" *50)

sys.stdout = orig_stdout
f.close( )



#step 9. 엑셀 형태로 저장하기
              
company = pd.DataFrame()

company['업종']=pd.Series(part2)
company['업체명']=c_name2
company['간단소개']=pd.Series(intro2)
company['블로그리뷰수']=pd.Series(blog_cnt4)
company['고객 별점수']=pd.Series(total_score2)
company['주소']=pd.Series(address2)
company['연락처']=pd.Series(tel2)
company['홈페이지']=pd.Series(homepage2)
company['언론소개']=pd.Series(tv2)

company.to_excel(fx_name)
print("엑셀로 저장한 경로와 파일명:",fx_name," 입니다")

# 엑셀파일 실행하기
import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
excel.ActiveWorkbook.Save()
excel.Visible=True
    
driver.close( )


연습문제 9-7 네이버에서 특정 키워드로 관련 장소나 업체정보 수집하기
1.크롤링할 키워드는 무엇입니까?(예:강남역 맛집): 인천맛집
2.크롤링 할 건수는 몇건입니까?: 500
3.파일이 저장될 경로만 쓰세요:(예:c:\temp\): c:\data\


--------------------------------------------------------------------------------
전체 검색 결과 건수 : 42226 건
실제 최종 검색 건수 500
--------------------------------------------------------------------------------


전체 25 페이지 중 현재 1 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 2 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 3 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 4 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 5 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 6 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 7 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 8 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 9 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 10 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 11 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 12 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 13 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 14 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 15 번째 페이지의 목록을 추출하는 중입니다


전체 25 페이지 중 현재 16 번째 페이지의 목록을 추출하는 

21 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 포그커피
2.간단소개: 포그카페 감성적이고 케이크가 맛있는곳
3.도로명주소: 인천 부평구 광장로4번길 8
4.연락처: 032-502-0830
5.업종: 테이크아웃커피
6.홈페이지: https://www.instagram.com/fog.coffee
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 526
----------------------------------------------------------------------


22 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 정가네김밥
2.간단소개:  
3.도로명주소: 인천 부평구 경원대로 1254
4.연락처: 032-502-6022
5.업종: 종합분식
6.홈페이지:  
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 0
----------------------------------------------------------------------


23 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 수사 뉴코아아울렛인천점
2.간단소개:  
3.도로명주소: 인천 남동구 인하로 485
4.연락처: 032-233-9923
5.업종: 일식,초밥뷔페
6.홈페이지:  
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 175
----------------------------------------------------------------------


24 번째 업체 정보입니다================ 
-------

46 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 홍대개미 부평점
2.간단소개: 덮밥 맛집 홍대개미 부평 문화의 거리 간단한 덮밥 요리 전문점
3.도로명주소: 인천 부평구 부평문화로80번길 19
4.연락처: 032-505-6006
5.업종: 덮밥
6.홈페이지: http://hdgaemi.com/
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 297
----------------------------------------------------------------------


47 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 잉글랜드왕돈까스
2.간단소개: 백종원의 3대천왕에도 나온 전통 경양식 돈까스
3.도로명주소: 인천 중구 우현로90번길 7
4.연락처: 032-772-7266
5.업종: 돈가스
6.홈페이지:  
7.언론소개: 수요미식회 196회19.02.27. 돈가스
8.평점:  
9.블로그 리뷰 수: 913
----------------------------------------------------------------------


48 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 선한정식
2.간단소개: 송도 맛집 베스트10  남다르신 한정식
3.도로명주소: 인천 연수구 능허대로267번길 59
4.연락처: 032-831-7767
5.업종: 한정식
6.홈페이지: http://www.entas.co.kr
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 135
-----------------------------

70 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: wknd Lounge
2.간단소개: 중구 송학동 동인천  자유공원 전망좋은 카페
3.도로명주소: 인천 중구 송학로 63
4.연락처: 070-7755-2106
5.업종: 카페,디저트
6.홈페이지: http://www.wknd.kr
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 506
----------------------------------------------------------------------


71 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 스시웨이 청라점
2.간단소개: 행복을 주는 청라초밥 스시
3.도로명주소: 인천 서구 청라라임로 65
4.연락처: 032-561-3704
5.업종: 초밥,롤
6.홈페이지: http://www.sushiway.co.kr
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 148
----------------------------------------------------------------------


72 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 아웃백스테이크하우스 송도점
2.간단소개:  
3.도로명주소: 인천 연수구 송도국제대로 157
4.연락처: 032-821-9951
5.업종: 패밀리레스토랑
6.홈페이지:  
7.언론소개:  
8.평점 4.5 / 5
9.블로그 리뷰 수: 122
----------------------------------------------------------------------



95 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 웨스턴스테이크
2.간단소개: 칠리치킨스테이크 저렴하고 맛있는 부평유명맛집
3.도로명주소: 인천 부평구 부평대로32번길 2
4.연락처: 032-522-5525
5.업종: 스테이크,립
6.홈페이지:  
7.언론소개: 생생정보통 786회14.01.06. 삼겹살스테이크
8.평점:  
9.블로그 리뷰 수: 143
----------------------------------------------------------------------


96 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 마시랑카페
2.간단소개: 인천 을왕리 영종도 바다가 보이는 카페
3.도로명주소: 인천 중구 마시란로 163
4.연락처: 032-746-5088
5.업종: 카페
6.홈페이지:  
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 803
----------------------------------------------------------------------


97 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 건강밥상 심마니
2.간단소개: 송도아울렛 맛집 자극적이지 않은 한식요리
3.도로명주소: 인천 연수구 해돋이로 161
4.연락처: 032-831-0831
5.업종: 백숙,삼계탕
6.홈페이지:  
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 373
----------------------------------------------------------------------


98 번째

...

143 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 애슐리W 부평역점
2.간단소개: 싱그러운 딸기축제가 한창 열리는 애슐리 부평역점
3.도로명주소: 인천 부평구 시장로 7
4.연락처: 032-252-1031
5.업종: 패밀리레스토랑
6.홈페이지:  
7.언론소개:  
8.평점 3.9 / 5
9.블로그 리뷰 수: 121
----------------------------------------------------------------------


144 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 마시안어부집
2.간단소개: 영종도 맛집 죽은조개안쓰는곳 어부집
3.도로명주소: 인천 중구 마시란로 79
4.연락처: 032-751-1013
5.업종: 조개요리
6.홈페이지:  
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 91
----------------------------------------------------------------------


145 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 불타는 주먹다짐
2.간단소개: 부평 고기집 초벌되서 나오는 주먹고기
3.도로명주소: 인천 부평구 경원대로 1422
4.연락처: 032-330-5824
5.업종: 돼지고기구이
6.홈페이지: https://firefist.modoo.at/
7.언론소개:  
8.평점 4.9 / 5
9.블로그 리뷰 수: 396
--------------------------------------------------------------------

167 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 아웃닭 부평점
2.간단소개: 치맥이 땡기는날은 치킨
3.도로명주소: 인천 부평구 시장로 20
4.연락처: 032-522-6221
5.업종: 치킨,닭강정
6.홈페이지:  
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 231
----------------------------------------------------------------------


168 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 김부삼 부평역점
2.간단소개: 부평 맵돈에서 매운돼지갈비
3.도로명주소: 인천 부평구 경원대로1403번길 6
4.연락처: 032-361-8015
5.업종: 돼지고기구이
6.홈페이지:  
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 16
----------------------------------------------------------------------


169 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 애슐리W 뉴코아 인천점
2.간단소개: 애슐리w 새우축제 즐기러 구월뉴코아점
3.도로명주소: 인천 남동구 인하로 485
4.연락처: 032-441-1033
5.업종: 패밀리레스토랑
6.홈페이지:  
7.언론소개:  
8.평점 4.4 / 5
9.블로그 리뷰 수: 56
----------------------------------------------------------------------


170 번째 업체 정보입니다================ 
-

192 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 마카롱라쥬아
2.간단소개: 마카롱 맛집 중 최근에 발견한 샤인머스캣 마카롱
3.도로명주소: 인천 부평구 부평문화로65번길 16
4.연락처: 070-7393-2332
5.업종: 카페,디저트
6.홈페이지: https://www.instagram.com/macaron_la_joie/
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 138
----------------------------------------------------------------------


193 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 카페라르고빌
2.간단소개: 강화도 바다 보이는 카페 largo 솜사탕
3.도로명주소: 인천 강화군 화도면 해안남로2845번길 25
4.연락처: 032-937-8847
5.업종: 카페,디저트
6.홈페이지: http://www.largoville.com
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 242
----------------------------------------------------------------------


194 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 신포가빈 구월본점
2.간단소개: 선수촌 맛집 신포가빈 고급스런 구월동 고기집 쪽갈비
3.도로명주소: 인천 남동구 선수촌공원로23번길 10-21
4.연락처: 032-472-3492
5.업종: 육류,고기요리
6.홈페이지: http://www.sinpogabin.com/
7.언론소개: 

216 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 화순반점
2.간단소개: 옛날느낌 간짜장 화끈한 고추짬뽕
3.도로명주소: 인천 동구 만석부두로 3
4.연락처: 032-762-4797
5.업종: 중식당
6.홈페이지:  
7.언론소개: 백종원의3대천왕 56회16.10.08. 간짜장/잡채
8.평점:  
9.블로그 리뷰 수: 78
----------------------------------------------------------------------


217 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 메리고라운드스테이크키친 송도커넬워크점
2.간단소개: 커넬워크 맛집으로 스테이크가 유명한 레스토랑
3.도로명주소: 인천 연수구 아트센터대로 107
4.연락처: 032-833-9141
5.업종: 패밀리레스토랑
6.홈페이지: http://merry-go-round.kr
7.언론소개:  
8.평점 4.2 / 5
9.블로그 리뷰 수: 461
----------------------------------------------------------------------


218 번째 업체 정보입니다================ 
----------------------------------------------------------------------
1.업체명: 봉추찜닭 인천부평점
2.간단소개: 부평찜닭 봉추찜닭 양념이 진짜 맛있는 찜닭
3.도로명주소: 인천 부평구 부평문화로72번길 20
4.연락처: 032-518-6981
5.업종: 찜닭
6.홈페이지:  
7.언론소개:  
8.평점:  
9.블로그 리뷰 수: 340
-----------------------------------

KeyboardInterrupt: 